# Extracción de las Características del corte de Audio

El siguiente codigo realiza la extraccion de las caracteristicas del corte de audio 'WAV' y el archivo 'TXT'

In [76]:
import librosa
from librosa import feature
from librosa import onset
import numpy as np
import pandas as pd
from glob import glob
import csv
from sklearn.model_selection import train_test_split

Estableciendo directorios de trabajo

In [77]:
data_dir = 'C:\\Data\\6_Clasificacion\\in'
data_out = 'C:\\Data\\6_Clasificacion\\out' 
audio_files = glob(data_dir + '/*.wav')
txt_files = glob(data_dir + '/*.txt')
#print(audio_files)

Funciones y arreglos

In [78]:
fn_list_i = [
 feature.chroma_stft,
 feature.chroma_cqt,
 feature.chroma_cens,
 feature.spectral_centroid,
 feature.spectral_bandwidth,
 feature.spectral_rolloff,
 feature.mfcc,
 feature.melspectrogram,
 onset.onset_detect
]
 
fn_list_ii = [
 feature.rmse,
 feature.zero_crossing_rate
]

header =[
 'Audio',
 'especie',
 'chroma_stft',
 'chroma_cqt',
 'chroma_cens',
 'spectral_centroid',
 'spectral_bandwidth',
 'spectral_rolloff',
 'mfcc',
 'melspectrogram',
 'onset_detect',
 'rmse',
 'zero_crossing_rate',
 'Corte',
 'startime',
 'finishtime',
 'id',
 'freqmin',
 'freqmax' 
]

def crear_archivo(ruta):
    archivo = open(ruta, 'w')
    return archivo
    
    
def leer_archivo (archivo):
    arch1 = open(archivo, 'r')
    contenido = arch1.read()
    return contenido
    
    
def escribir_archivo(archivo, texto):
    archivo.write(texto)

def get_feature_vector(file, etiqueta, y,sr): 
   feat_vect = [file]
   feat_vect_i = [ np.mean(funct(y,sr)) for funct in fn_list_i]
   feat_vect_ii = [ np.mean(funct(y)) for funct in fn_list_ii]
   feat_vect_iii = [(etiqueta)]
   feature_vector = feat_vect + feat_vect_iii + feat_vect_i + feat_vect_ii 
#print(feature_vector)
   return feature_vector


def get_features_texto(file_txt):
    cont = leer_archivo(file_txt)
    #cont = cont.replace("'","")
    lista = cont.split(",") 
    datos = [str(lista[0])] + [float(lista[1])] + [float(lista[2])] + [str(lista[3])] + [float(lista[4])] + [float(lista[5])]
    #print(datos)
    return datos

Generando Vector de Trabajo

In [79]:
audios_feat = []
for file in audio_files:
   y, sr = librosa.load(file,sr=None)
   #print(file)
   filename = file.replace(data_dir + '\\', '')
   etiqueta = 'para_clasificar'
   file_txt = file[0:len(file)-4]+'.txt'
   text_vector = get_features_texto(file_txt)
   feature_vector = get_feature_vector(filename, etiqueta, y, sr)
   audios_feat.append(feature_vector + text_vector) 


Definiendo DATAFRAME y archivo CSV

In [80]:
norm_output = data_out+'\\df_'+etiqueta+'.csv'
norm_output = norm_output.replace('//', '/')
df = pd.DataFrame(audios_feat)
df.columns = header 
tiempo=df['finishtime']-df['startime']
df=df.assign(duration=tiempo)
del df['startime']
del df['finishtime']
del df['Audio']
del df['Corte']
del df['id']
del df['especie']
df.to_csv(norm_output)

Aqui termina la extracion de las caracteristicas

datos = 'C:\\Data\\2_Preprocesamiento\\Pruebas_8\\Todos.csv'
dataframe = pd.read_csv(datos,sep=',')

del dataframe['spectral_flatness']
del dataframe['feature.tonnetz']
del dataframe['pitch']

dataframe.to_csv('C:\\Data\\2_Preprocesamiento\\Pruebas_9\\Todos.csv')